This notebook will be mainly used for the Coursera Capstone project 

In [26]:
import pandas as pd 
import numpy as np 

In [27]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Scraping Wikipedia for Toronto Neighborhoods

In [28]:
!pip install beautifulsoup4

In [29]:
import wikipedia
from bs4 import BeautifulSoup

Use the wikipedia package to scrape the website. Parse the html content with Beautifulsoup by looking for tables. 

In [60]:
page = wikipedia.page('List_of_postal_codes_of_Canada:_M')

html_content = page.html()

soup = BeautifulSoup(html_content)

table = soup.find_all('table')[0]

Scanning through the table, build a DataFrame containing the Neighborhood data

In [101]:
toronto_df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

In [102]:
for r, row in enumerate(table.find_all('tr')):
    col = []
    for c, column in enumerate(row.find_all('td')):
        col.append(column.get_text().strip())
    if col:
        toronto_df.loc[r] = col

Convert 'Not assigned" to NaNs for easier pandas processing

In [103]:
toronto_df = toronto_df.replace('Not assigned',np.nan)

Drop entries where Borough is not assigned 

In [104]:
toronto_df = toronto_df[~toronto_df['Borough'].isna()].reset_index(drop=True)

Where neighborhood is not assigned, fill field with Borough name 

In [105]:
toronto_df.loc[:,'Neighborhood'] = toronto_df.loc[:,'Neighborhood'].fillna(toronto_df.Borough)

As suggested in assignment some postal codes may occur more than once, however it seems like the wikipedia article has been updated since. 
No duplicate PCs were found:

In [106]:
toronto_df.loc[toronto_df.PostalCode == 'M5A']

PostalCode           Borough               Neighborhood
2        M5A  Downtown Toronto  Regent Park, Harbourfront

In [107]:
(toronto_df.groupby('PostalCode').count() > 1).sum()

Borough         0
Neighborhood    0
dtype: int64

In [108]:
toronto_df.head()

PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [109]:
toronto_df.to_csv('toronto_neighborhoods.csv')

In [110]:
toronto_df.shape

(103, 3)